In [1]:
import pandas as pd # for using dataframe

[예측 절차]

1. 데이터 프레임 생성
2. 예측해야할 데이터들 선정(데이터 프레임에서 np.nan인 값들 추출)
3. 유사도 정의
4. 예측하기

In [2]:
critics = {
    '조용필': {
        '택시운전사': 2.5,
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '넘버3': 3.5,
        '사랑과전쟁': 2.5,
        '세계대전': 3.0,
    },
    'BTS': {
        '택시운전사': 1.0,
        '겨울왕국': 4.5,
        '리빙라스베가스': 0.5,
        '넘버3': 1.5,
        '사랑과전쟁': 4.5,
        '세계대전': 5.0,
    },
    '강감찬': {
        '택시운전사': 3.0,
        '겨울왕국': 3.5,
        '리빙라스베가스': 1.5,
        '넘버3': 5.0,
        '세계대전': 3.0,
        '사랑과전쟁': 3.5,
    },
    '을지문덕': {
        '택시운전사': 2.5,
        '겨울왕국': 3.0,
        '넘버3': 3.5,
        '세계대전': 4.0,
    },
    '김유신': {
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '세계대전': 4.5,
        '넘버3': 4.0,
        '사랑과전쟁': 2.5,
    },
    '유성룡': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '리빙라스베가스': 2.0,
        '넘버3': 3.0,
        '세계대전': 3.5,
        '사랑과전쟁': 2.0,
    },
    '이황': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '세계대전': 3.0,
        '넘버3': 5.0,
        '사랑과전쟁': 3.5,
    },
    '이이': {'겨울왕국': 4.5, '사랑과전쟁': 1.0,
             '넘버3': 4.0},
}

In [3]:
# 1. 데이터 프레임 생성
df = pd.DataFrame(critics)

In [4]:
df

,조용필,BTS,강감찬,을지문덕,김유신,유성룡,이황,이이
겨울왕국,3.5,4.5,3.5,3.0,3.5,4.0,4.0,4.5
넘버3,3.5,1.5,5.0,3.5,4.0,3.0,5.0,4.0
리빙라스베가스,3.0,0.5,1.5,NaN,3.0,2.0,NaN,NaN
사랑과전쟁,2.5,4.5,3.5,NaN,2.5,2.0,3.5,1.0
세계대전,3.0,5.0,3.0,4.0,4.5,3.5,3.0,NaN
택시운전사,2.5,1.0,3.0,2.5,NaN,3.0,3.0,NaN


In [5]:
# 2. 예측해야할 데이터들 선정(데이터 프레임에서 np.nan인 값들 추출)
lstNone = [] # 행, 열 기준
for col in df.columns:
    for row in df.index:
        if df.loc[row, col].__str__() == 'nan':
            lstNone.append((row, col))

In [6]:
lstNone

[('리빙라스베가스', '을지문덕'),
 ('사랑과전쟁', '을지문덕'),
 ('택시운전사', '김유신'),
 ('리빙라스베가스', '이황'),
 ('리빙라스베가스', '이이'),
 ('세계대전', '이이'),
 ('택시운전사', '이이')]

In [7]:
# 3. 유사도 정의
def sim_distance(df,user1, user2):
    """
    입력
      df: 영화에 대한 데이터 프레임, 행인덱스는 영화, 열인덱스는 사람으로 되어 있음
      user1: 첫 번째 사람(str 타입)
      user2: 두 번째 사람(str 타입)
    출력
      유클리디안 거리 기반으로 유사도 측정값을 반환
    """
    s = df[user1] - df[user2]
    s.dropna(inplace=True)
    ans = s.apply(lambda x:x**2).sum()
    return 1/(1+ans**0.5)

In [8]:
# 4. 예측하기
def predict(df, row, col):
    """
    입력:
      df: 영화에 대한 데이터 프레임, 행인덱스는 영화, 열인덱스는 사람으로 되어 있음
      row: 예측하려는 영화 평점의 행 인덱스(영화)
      col: 예측하려는 영화 평점의 열 인덱스(사람)
    출력:
      sum(col과 각 사람과의 유사도*각 사람의 영화 평점)/sum(col과 각 사람과의 유사도)
    """
    pred = 0
    simLst = []
    for i_col in df.columns:
        if (col == i_col) or (df.loc[row, i_col].__str__() == 'nan'):
            continue
        sim = sim_distance(df, col, i_col)
        rate = df.loc[row, i_col]
        pred += sim*rate
        simLst.append(sim)
    ans = pred/sum(simLst)
    return ans

In [11]:
ans = {}
for cur in lstNone:
    ans[cur] = predict(df, cur[0], cur[1])

In [12]:
ans

{('리빙라스베가스', '을지문덕'): 2.2323640321235456,
 ('사랑과전쟁', '을지문덕'): 2.6297516115569564,
 ('택시운전사', '김유신'): 2.5907639312248794,
 ('리빙라스베가스', '이황'): 2.03440847815843,
 ('리빙라스베가스', '이이'): 2.1888145663244445,
 ('세계대전', '이이'): 3.6796317295600414,
 ('택시운전사', '이이'): 2.597239917149355}

In [20]:
d = {}
for i in ans:
    if d.get(i[1]):
        if ans[i] > d[i[1]][1]:
            d[i[1]] = i[0], ans[i]
    else:
        d[i[1]] = i[0], ans[i]

In [21]:
d

{'을지문덕': ('사랑과전쟁', 2.6297516115569564),
 '김유신': ('택시운전사', 2.5907639312248794),
 '이황': ('리빙라스베가스', 2.03440847815843),
 '이이': ('세계대전', 3.6796317295600414)}